In [29]:
import argparse
from dataset import load

import numpy as np
import torch as th
import torch.nn as nn

from eval import label_classification, eval_unbiasedness_movielens
import warnings

warnings.filterwarnings('ignore')


def count_parameters(model):
    return sum([np.prod(p.size()) for p in model.parameters() if p.requires_grad])


parser = argparse.ArgumentParser()
parser.add_argument('--dataname', type=str, default='movielens')
parser.add_argument('--gpu', type=int, default=1)
parser.add_argument('--split', type=str, default='random')
parser.add_argument('--debias_method', type=str, default='uge-c', choices=['uge-r', 'uge-w', 'uge-c', 'none'], help='debiasing method to apply')
parser.add_argument('--debias_attr', type=str, default='age', help='sensitive attribute to be debiased')
parser.add_argument('--reg_weight', type=float, default=2, help='weight for the regularization based debiasing term')  

parser.add_argument('--epochs', type=int, default=200, help='Number of training periods.')
parser.add_argument('--lr', type=float, default=0.001, help='Learning rate.')
parser.add_argument('--wd', type=float, default=1e-5, help='Weight decay.')
parser.add_argument('--temp', type=float, default=1.0, help='Temperature.')

parser.add_argument('--act_fn', type=str, default='relu')

parser.add_argument("--hid_dim", type=int, default=256, help='Hidden layer dim.')
parser.add_argument("--out_dim", type=int, default=256, help='Output layer dim.')

parser.add_argument("--num_layers", type=int, default=2, help='Number of GNN layers.')
parser.add_argument('--der1', type=float, default=0.2, help='Drop edge ratio of the 1st augmentation.')
parser.add_argument('--der2', type=float, default=0.2, help='Drop edge ratio of the 2nd augmentation.')
parser.add_argument('--dfr1', type=float, default=0.2, help='Drop feature ratio of the 1st augmentation.')
parser.add_argument('--dfr2', type=float, default=0.2, help='Drop feature ratio of the 2nd augmentation.')

args = parser.parse_args("")

if args.gpu != -1 and th.cuda.is_available():
    args.device = 'cuda:{}'.format(args.gpu)
else:
    args.device = 'cpu'

# Step 1: Load hyperparameters =================================================================== #
lr = args.lr
hid_dim = args.hid_dim
out_dim = args.out_dim

num_layers = args.num_layers
act_fn = ({'relu': nn.ReLU(), 'prelu': nn.PReLU()})[args.act_fn]

drop_edge_rate_1 = args.der1
drop_edge_rate_2 = args.der2
drop_feature_rate_1 = args.dfr1
drop_feature_rate_2 = args.dfr2

temp = args.temp
epochs = args.epochs
wd = args.wd
debias_method = args.debias_method

# Step 2: Prepare data =================================================================== #
if debias_method in ['uge-w', 'uge-c']:
    dataset = '{}_debias_{}'.format(args.dataname, args.debias_attr)
else:
    dataset = args.dataname

graph = load(dataset)
in_dim = graph.ndata['feat'].shape[1]

Precomputed weights for weighting-based debiasing UGE-W Loaded
Creating DGL graph...
Finished data loading and preprocessing.
  NumNodes: 9992
  NumEdges: 2010410
  NumFeats: 18


In [30]:
from dataset import SENSITIVE_ATTR_DICT  # predefined sensitive attributes for different datasets
from dataset import DATA_FOLDER
import pandas as pd

SENSITIVE_ATTR_DICT = {
    'movielens': ['gender', 'occupation', 'age'],
    'pokec': ['gender', 'region', 'AGE'],
    'pokec-z': ['gender', 'region', 'AGE'],
    'pokec-n': ['gender', 'region', 'AGE'],
}
# Group nodes
debias_attr = args.debias_attr
attribute_list = SENSITIVE_ATTR_DICT[args.dataname]

non_sens_attr_ls = [attr for attr in attribute_list if attr!=debias_attr]
non_sens_attr_idx = [i for i in range(len(attribute_list)) if attribute_list[i]!=debias_attr]

attribute_file = '{}/{}_node_attribute.csv'.format(DATA_FOLDER, args.dataname)
node_attributes = pd.read_csv(attribute_file)

attr_comb_groups = node_attributes.groupby(attribute_list)
nobias_comb_groups = node_attributes.groupby(non_sens_attr_ls)

attr_comb_groups_map = {tuple(group[1].iloc[0]):list(group[1].index) 
                        for group in attr_comb_groups}
nobias_attr_comb_groups_map = {tuple(group[1].iloc[0][non_sens_attr_ls]):list(group[1].index) 
                            for group in nobias_comb_groups}

print ('Group finished.')
print ('  attr_comb_group_num:', len(attr_comb_groups_map.keys()))
print ('  nobias_attr_comb_group_num:', len(nobias_attr_comb_groups_map.keys()))

Group finished.
  attr_comb_group_num: 242
  nobias_attr_comb_group_num: 43


In [31]:
def map_tuple(x, index_ls):
  return tuple([x[idx] for idx in index_ls])

def mem_eff_matmul_mean(mtx1, mtx2):
  mtx1_rows = list(mtx1.shape)[0]
  if mtx1_rows <= 1000:
    return th.mean(th.matmul(mtx1, mtx2))
  else:
    value_sum = 0
    for i in range(mtx1_rows // 1000):
      value_sum += th.sum(th.matmul(mtx1[i*1000:(i+1)*1000, :], mtx2))
    if mtx1_rows % 1000 != 0:
      value_sum += th.sum(th.matmul(mtx1[(i+1)*1000:, :], mtx2))
    return value_sum / (list(mtx1.shape)[0] * list(mtx2.shape)[1])

In [32]:
'weight' in graph.edata

True

In [33]:
import dgl

def aug(graph, feat_drop_rate, edge_mask_rate):
    n_node = graph.num_nodes()
    x = graph.ndata['feat']

    edge_mask = mask_edge(graph, edge_mask_rate)
    feat = drop_feature(x, feat_drop_rate)

    src = graph.edges()[0]
    dst = graph.edges()[1]

    nsrc = src[edge_mask]
    ndst = dst[edge_mask]
    weights = graph.edata['weight'][edge_mask]

    ng = dgl.graph((nsrc, ndst), num_nodes=n_node)
    ng.edata['weight'] = weights
    ng = ng.add_self_loop()

    return ng, feat

def drop_feature(x, drop_prob):
    drop_mask = th.empty((x.size(1),),
                        dtype=th.float32,
                        device=x.device).uniform_(0, 1) < drop_prob
    x = x.clone()
    x[:, drop_mask] = 0

    return x

def mask_edge(graph, mask_prob):
    E = graph.num_edges()

    mask_rates = th.FloatTensor(np.ones(E) * mask_prob)
    masks = th.bernoulli(1 - mask_rates)
    mask_idx = masks.nonzero().squeeze(1)
    return mask_idx

dr = 0.2
aug(graph, feat_drop_rate=dr, edge_mask_rate=dr)

(Graph(num_nodes=9992, num_edges=1617948,
       ndata_schemes={}
       edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)}),
 tensor([[1., 0., 0.,  ..., 1., 1., 0.],
         [1., 0., 0.,  ..., 1., 1., 1.],
         [1., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.]]))

In [42]:
import importlib
import model
importlib.reload(model)
import random

dr = 0.2
# Step 3: Create emb_model =================================================================== #
emb_model = model.Grace(in_dim, hid_dim, out_dim, num_layers, act_fn, temp)
emb_model = emb_model.to(args.device)
print(f'# params: {count_parameters(emb_model)}')

optimizer = th.optim.Adam(emb_model.parameters(), lr=lr, weight_decay=wd)

# Step 4: Training =======================================================================
for epoch in range(args.epochs):
    emb_model.train()
    optimizer.zero_grad()
    graph1, feat1 = aug(graph, feat_drop_rate=dr, edge_mask_rate=dr)
    graph2, feat2 = aug(graph, feat_drop_rate=dr, edge_mask_rate=dr)

    graph1 = graph1.to(args.device)
    graph2 = graph2.to(args.device)

    feat1 = feat1.to(args.device)
    feat2 = feat2.to(args.device)

    loss = emb_model(graph1, graph2, feat1, feat2, useWeight=True)
    
    # UGE-R
    if debias_method in ['uge-r', 'uge-c']:
        h1 = emb_model.encoder(graph1, feat1)
        h2 = emb_model.encoder(graph2, feat2)
        regu_loss = 0
        scr_groups = random.sample(list(attr_comb_groups_map.keys()), 100)  
        dst_groups = random.sample(list(attr_comb_groups_map.keys()), 100)
        nobias_scr_groups = [map_tuple(group, non_sens_attr_idx) for group in scr_groups]
        nobias_dst_groups = [map_tuple(group, non_sens_attr_idx) for group in dst_groups]

        for group_idx in range(len(scr_groups)):
            for view in [h1, h2]:
                scr_group_nodes = attr_comb_groups_map[scr_groups[group_idx]]
                dsc_group_nodes = attr_comb_groups_map[dst_groups[group_idx]]
                
                scr_node_embs = view[scr_group_nodes]
                dsc_node_embs = view[dsc_group_nodes]
                aver_score = mem_eff_matmul_mean(scr_node_embs, dsc_node_embs.T)

                nobias_scr_group_nodes = nobias_attr_comb_groups_map[nobias_scr_groups[group_idx]]
                nobias_dsc_group_nodes = nobias_attr_comb_groups_map[nobias_dst_groups[group_idx]]
                nobias_scr_node_embs = view[nobias_scr_group_nodes]
                nobias_dsc_node_embs = view[nobias_dsc_group_nodes]
                nobias_aver_score = mem_eff_matmul_mean(nobias_scr_node_embs, nobias_dsc_node_embs.T)

                regu_loss += th.square(aver_score - nobias_aver_score)
            
        print(f"Epoch={epoch:03d}, loss: {loss.item():.2f}, regu_loss: {regu_loss.item():.2f}")

        loss += args.reg_weight * regu_loss / 1000
    
    loss.backward()
    optimizer.step()

    print(f'Epoch={epoch:03d}, loss={loss.item():.4f}')

# Step 5: Linear evaluation ============================================================== #
print("=== Final ===")

graph = graph.add_self_loop()
graph = graph.to(args.device)
embeds = emb_model.get_embedding(graph, graph.ndata['feat'].to(args.device))



# params: 272640
Epoch=000, loss: 9.85, regu_loss: 61.40
Epoch=000, loss=11.0743
Epoch=001, loss: 9.80, regu_loss: 20.57
Epoch=001, loss=10.2113
Epoch=002, loss: 9.79, regu_loss: 6.85
Epoch=002, loss=9.9246
Epoch=003, loss: 9.75, regu_loss: 2.01
Epoch=003, loss=9.7904
Epoch=004, loss: 9.77, regu_loss: 0.71
Epoch=004, loss=9.7833
Epoch=005, loss: 9.67, regu_loss: 0.98
Epoch=005, loss=9.6945
Epoch=006, loss: 9.57, regu_loss: 0.30
Epoch=006, loss=9.5752
Epoch=007, loss: 9.47, regu_loss: 0.45
Epoch=007, loss=9.4816
Epoch=008, loss: 9.39, regu_loss: 0.62
Epoch=008, loss=9.3977
Epoch=009, loss: 9.39, regu_loss: 1.04
Epoch=009, loss=9.4154
Epoch=010, loss: 9.39, regu_loss: 1.28
Epoch=010, loss=9.4110
Epoch=011, loss: 9.41, regu_loss: 1.12
Epoch=011, loss=9.4339
Epoch=012, loss: 9.39, regu_loss: 1.41
Epoch=012, loss=9.4142
Epoch=013, loss: 9.38, regu_loss: 0.80
Epoch=013, loss=9.3991
Epoch=014, loss: 9.37, regu_loss: 1.78
Epoch=014, loss=9.4095
Epoch=015, loss: 9.38, regu_loss: 1.69
Epoch=015,

In [43]:
from eval import label_classification, eval_unbiasedness_movielens
'''Evaluation Embeddings  '''
# label_classification(embeds, graph.ndata['label'], graph.ndata['train_mask'], graph.ndata['test_mask'], split=args.split)
res = eval_unbiasedness_movielens('movie', embeds.cpu())

loading data ...
Unbiasedness evaluation (predicting attribute)
-- micro-f1 when predicting gender: 0.5490384615384616
-- micro-f1 when predicting age: 0.23557692307692307
-- micro-f1 when predicting occupation: 0.032692307692307694
Utility evaluation (link prediction)
-- ndcg of link prediction: 0.32871755543670783


In [36]:
res

{'unbiasedness': {'gender': 0.33942307692307694,
  'age': 0.038461538461538464,
  'region': 0.0,
  'occupation': 0.11634615384615385},
 'utility': 0.01643908590532295}

In [37]:
res['utility']

0.01643908590532295

In [38]:
res = eval_unbiasedness_movielens('movie', th.randn_like(embeds).cpu())

loading data ...
Unbiasedness evaluation (predicting attribute)
-- micro-f1 when predicting gender: 0.5134615384615384
-- micro-f1 when predicting age: 0.14038461538461539
-- micro-f1 when predicting occupation: 0.045192307692307684
Utility evaluation (link prediction)
-- ndcg of link prediction: 0.020967456252946947


In [39]:
import sys
sys.path.append(os.path.join('../..'))
import Utils.Export as Export

results = {
  "dataname": args.dataname,
  "epochs": args.epochs,
  "debias_method": "random",
  "debias_attr": args.debias_attr,
  "reg_weight": args.reg_weight,
  "temp": args.temp,
  "der1": args.der1,
  "der2": args.der2,
  "dfr1": args.dfr1,
  "dfr2": args.dfr2,
  "gender_f1m": res['unbiasedness']['gender'],
  "age_f1m": res['unbiasedness']['age'],
  "occupation_f1m": res['unbiasedness']['occupation'],
  "link_ndcg": res['utility'],
}

Export.saveData('./results.csv', results)